In [35]:
# -*- coding: utf-8 -*-
# imports
import pandas as pd
import requests
import re
import os
from yaml import safe_load
from bs4 import BeautifulSoup as bs
import random


"""
sdg-csv-data-filler is the first module in a data pipeline to take
data from the SDG data repo and make it exportable as CSVW.
"""

# setting paths to directories and files
remote_data_url = "https://github.com/ONSdigital/sdg-data/tree/develop/data"
cwd = os.getcwd()
data_path = os.path.join(cwd, 'data')
out_path = os.path.join(cwd, 'out')
gap_filler_yam_path = (os.path.join
                       ("substitutions",
                        "gap_filler.yaml"))
header_mapping_yam_path = (os.path.join
                           ("substitutions",
                            "header_mapping.yaml"))

In [4]:
def get_mapping_dicts(path_to_yaml):
    """
    Loads dictionaries for the gap filling and mapping of column names 
    from locally stored .yaml files
    
        Parameters:
            gap_filler_yaml (string): Path to the yaml file storing the values
                to fill gaps with in each column
            header_mapping_yaml (string): Path to the yaml file storing the 
                names to change headers to for each column
        Returns:
            dict: dict_from_yam
            
    """
    with open(path_to_yaml) as file:
        dict_from_yam = safe_load(file)
    
    return dict_from_yam

In [5]:
def find_csv_urls(url):
    """
    Provided with a data folder URL, this function finds the URLS
    of the CSV files within the folder. A generator is yielded with
    the links of all files in the folder.
        Parameters:
            url (string): the URL of the repo/folder which contains
                the CSV files to be captured
        Yields:
            string: generator, the next URL for the CSV file in the 
            remote data folder 
    """
    page = requests.get(url, verify=False) #proxies=proxies
    soup = bs(page.text, 'html.parser')
    csv_link_pattern = r"\/ONSdigital\/sdg-data\/blob\/develop\/data\/indicator_\d-\d{1,2}-\d{1,2}\.csv"
    to_repl_pattern = r"\/sdg-data\/blob\/develop"
    replacement_pattern = "/sdg-data/develop"
    list_of_links = []
    for link in soup.findAll('a', attrs={'href': re.compile(csv_link_pattern)}):
        link = link.get('href')
        link = re.sub(to_repl_pattern, replacement_pattern, link)
        yield ("https://raw.githubusercontent.com"+link)
        # Alternative to return a list
        # list_of_links.append("https://raw.githubusercontent.com"+link)
    # return list_of_links

In [6]:
def csvs_to_pandas(url):
    """
    Provided with a URL of a file, the fucntion will check if the CSV
    is populated and if not empty return a Pandas dataframe of the CSV
        Parameters:
            url (string): the URL of a CSV file to be captured
        Returns:
            pd.DataFrame: a Pandas dataframe of the CSV
    """
    if "no data for this indicator yet" in str(bs(requests.get(url).text)):
        return None
    else:
        return pd.read_csv(url)

In [7]:
def csvsample_to_pandas(path_to_file, pct=1.0):
    """
    A function to create a sample extract of a csv as a dataframe
    
        Parameters:
            path_to_file (string): full path to csv file
            p (float): decimal amount of lines to extract
            
        Returns:
            pd.Dataframe
            """
    p = pct/100  
    # keep the header, then take only 1% of lines
    # if random from [0,1] interval is greater than 0.01 the row will be skipped
    df = pd.read_csv(
             path_to_file,
             header=0, 
             skiprows=lambda i: i>0 and random.random() > p)
    return df


In [8]:
def fill_gaps(pd_df, gap_filler_dict):
    """
    Given a Pandas dataframe and a dictionary containing the column names
    the correct 'fillers' for each column, this function will fill
    each column with the correct values when empty cells are found.
        Parameters:
            pd_df (pd.Dataframe): the variable to which the dataframe 
                containing the csv data is assigned
            gap_filler_dict (dict): a dictionary with column name and value 
                to fill gaps as key value pairs, e.g.
                {"Age":"All","Sex":"T"}
        Returns:
            pd.Dataframe: A dataframe with all cells full"""
    df = pd_df.fillna(gap_filler_dict, axis=1)
    return df

In [9]:
def standardise_cell_values(pd_df, dict_of_nonstandard_standard):
    """
    Maps non-standard values e.g. "Males" to standard values like "M".
    Mapping is carried out on a column-specific basis.
    """
    df = (pd_df.replace
          (to_replace=dict_of_nonstandard_standard,
          value=None))
    return df

In [28]:
def write_csv(df, out_path):
    """
    Converts a Pandas dataframe to CSV and writes it out to a local folder.
        Parameters:
            pd_df (pd.Dataframe): The pandas data frame of the data
            path (string): the path of the local "out" folder
        Returns:
            Boolean: True is written, False if not written """ 
    status = True

    # If the csv dir isn't there, make it
    csv_dir = out_path
    if not os.path.exists(csv_dir):
        os.makedirs(csv_dir, exist_ok=True)

    try:
        df.to_csv(csv_dir, index=False)
    except Exception as e:
        print(e)
        return False

    return status                   


In [29]:
# def entry_point(data_url):
#     urls_gen = find_csv_urls(data_url)
#     gap_filler_dict = (get_mapping_dicts(gap_filler_yam_path) 
#     header_mapping_dict = (get_mapping_dicts(header_mapping_yam_path))

#     for _url in urls_gen:
#         data_name = extract_name(_url)
#         df = csvs_to_pandas(_url)
#         if not df:
#             continue
#         df = fill_gaps(df, gap_filler_dict)
#         df = standardise_headers(df)
#         write_csv(df, out_path)

# if __name__ == "__main__":
#     entry_point(data_url=remote_data_url)

In [30]:
def delete_random_values(df, holes=20):
    """
    Smashes holes in your dataframe to the approximate number that you
    request (randint might choose the same cell twice)
    """
    for i in range(holes):
        row = random.randint(1, df.shape[0]-1)
        col = random.randint(0, df.shape[1]-1)
        df.iloc[row, col] = float('nan')
    return df



In [42]:
test_yam_path = os.path.join(os.getcwd(), "substitutions", "samp_filler.yaml")
csv_path = os.path.join(os.getcwd(), "data", "indicator_9-1-1.csv")
dict_of_nonstandard_standard_path = (os.path.join(os.getcwd(),
                                                  "substitutions",
                                                  "test_dict_of_nonstandard_standard.yaml"))

def proof_of_concept(csv_path,
                     ns=dict_of_nonstandard_standard_path, 
                     path=test_yam_path,
                     out_path=out_path,
                     file_name="test_df.csv"):
    import ipdb; ipdb.set_trace()
    # Creating a sample df. This is in place of csvs_to_pandas
    samp_df = csvsample_to_pandas(path_to_file=csv_path, pct=1)
    # Testing get_mapping_dicts func. Creating filler dictioary
    samp_filler_dict = get_mapping_dicts(path)
    # Creating dictionary to map non-standard terms with standard ones. 
    nonstandard_standard = get_mapping_dicts(ns)
    # Creating gaps in the data (this will not be used in production)
    holey_df = delete_random_values(samp_df)
    # Testing that the filler dict works
    refilled_df = holey_df.fillna(value=samp_filler_dict)
    # Testing that standardise dict dills gaps as expected
    samp_df = standardise_cell_values(refilled_df, nonstandard_standard)
    #Writing the df to csv locally. 
    was_written = write_csv(samp_df, os.path.join(out_path, file_name))
    return samp_df, was_written

poc_df, was_written = proof_of_concept(csv_path, path=test_yam_path)

> <ipython-input-42-23159a82f037>(14)proof_of_concept()
     13     # Creating a sample df. This is in place of csvs_to_pandas
---> 14     samp_df = csvsample_to_pandas(path_to_file=csv_path, pct=1)
     15     # Testing get_mapping_dicts func. Creating filler dictioary

ipdb> c
[Errno 13] Permission denied: 'D:\\repo\\sdg_csvw\\sdg-csv-data-filler\\out\\test_df.csv'


In [43]:
was_written

False